# Overview
Tracing is a powerful tool for understanding the behavior of your LLM application. Phoenix has best-in-class tracing, regardless of what framework you use, and has first-class instrumentation for a variety of frameworks (LlamaIndex, LangChain, DSPy),  SDKs (OpenAI, Bedrock, Mistral, Vertex), and Languages (Python, Javascript). You can also manually instrument your application using the OpenTelemetry SDK.

This example will walk you through how to use Phoenix to trace OpenAI requests.

# Install Dependencies
Let's start by installing the necessary dependencies.

In [1]:
!pip install -q "arize-phoenix>=4.29.0"

# Launch Phoenix

You have a few options for how to start a Phoenix app. We're using a cloud instance for this tutorial, but you can launch Phoenix in multiple different ways. If you don't want to sign up for a cloud instance, you can start a Phoenix app in your notebook environment or via docker.

In [2]:
# Check if PHOENIX_API_KEY is present in the environment variables.
# If it is, we'll use the cloud instance of Phoenix. If it's not, we'll start a local instance.
# A third option is to connect to a docker or locally hosted instance.
# See https://docs.arize.com/phoenix/setup/environments for more information.

import os

if "PHOENIX_API_KEY" in os.environ:
    os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

else:
    import phoenix as px

    px.launch_app().view()

/opt/anaconda3/envs/phoenix/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
📺 Opening a view to the Phoenix app. The app is running at http://localhost:6006/


Now that we have Phoenix configured, we can register that instance with OpenTelemetry, which will allow us to collect traces from our application here.

In [3]:
from phoenix.otel import register

tracer_provider = register()

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



# Instrument your application

Now we need to indicate which methods and attributes we want to trace. Phoenix has a number of built-in tracers for popular frameworks, and provides tools to manually instrument your application if needed. See [here for a list of integrations](https://docs.arize.com/phoenix/tracing/integrations-tracing)

Here we're using OpenAI, so we'll the built-in OpenAI instrumentor we provide.

In [4]:
!pip install -q openinference-instrumentation-openai openai

I0000 00:00:1726019136.795300  327237 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [5]:
from openinference.instrumentation.openai import OpenAIInstrumentor

OpenAIInstrumentor().instrument(tracer_provider=tracer_provider, skip_dep_check=True)

# Use OpenAI as normal

From here we can use OpenAI as normal. All of our requests will be traced and reported to Phoenix automatically.

In [6]:
# Add OpenAI API Key
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [11]:
import openai

client = openai.OpenAI()
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "Write a haiku."}],
)
print(response.choices[0].message.content)

Golden leaves cascade,
Whispers of autumn's embrace—
Nature's quiet song.


In [12]:
# Now following tutorial: https://docs.arize.com/phoenix/inferences/phoenix-inferences
import pandas as pd

train_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/cv/human-actions/human_actions_training.parquet"
)

In [19]:
train_df.head()

,prediction_id,prediction_ts,url,image_vector,actual_action,predicted_action
0,595d87df-5d50-4d60-bc5f-3ad1cc483190,1.655757e+09,https://storage.googleapis.com/arize-assets/fi...,"[0.26720312, 0.02652928, 0.0, 0.028591828, 0.0...",drinking,drinking
1,37596b85-c007-4e4f-901d-b87e5297d4b8,1.655757e+09,https://storage.googleapis.com/arize-assets/fi...,"[0.08745878, 0.0, 0.16057675, 0.036570743, 0.0...",fighting,fighting
2,b048d389-539a-4ffb-be61-2f4daa52e700,1.655757e+09,https://storage.googleapis.com/arize-assets/fi...,"[0.9822482, 0.0, 0.037284207, 0.017358225, 0.2...",clapping,clapping
3,3e00c023-49b4-49c2-9922-7ecbf1349c04,1.655757e+09,https://storage.googleapis.com/arize-assets/fi...,"[0.028404092, 0.063946, 1.0448836, 0.65191674,...",fighting,fighting
4,fb38b050-fb12-43af-b27d-629653b5df86,1.655758e+09,https://storage.googleapis.com/arize-assets/fi...,"[0.06121698, 0.5172761, 0.50730985, 0.5771937,...",sitting,sitting


In [13]:
# Define Schema to indicate which columns in train_df should map to each field
train_schema = px.Schema(
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="predicted_action",
    actual_label_column_name="actual_action",
    embedding_feature_column_names={
        "image_embedding": px.EmbeddingColumnNames(
            vector_column_name="image_vector",
            link_to_data_column_name="url",
        ),
    },
)

In [17]:
train_ds = px.Inferences(dataframe=train_df, schema=train_schema, name="training")

In [18]:
session = px.launch_app(primary=train_ds)

WARNI [phoenix.session.session] Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


I0000 00:00:1726021546.772817  352978 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
/opt/anaconda3/envs/phoenix/lib/python3.12/site-packages/phoenix/server/api/types/EmbeddingDimension.py:425: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prediction_id=dataset[PREDICTION_ID][row_id],
/opt/anaconda3/envs/phoenix/lib/python3.12/site-packages/phoenix/server/api/types/EmbeddingDimension.py:426: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  link_to_data=dataset[self.dimension.link_to_data][row_id],
/opt/anaconda3/envs/phoenix/lib/python3.12/site-packages/phoenix/server/api/types

# Add comparison data

In [21]:
#Add comparison data (visualize drift, conduct A/B tests of production data against training set
prod_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/cv/human-actions/human_actions_training.parquet"
)

prod_df.head()

,prediction_id,prediction_ts,url,image_vector,actual_action,predicted_action
0,595d87df-5d50-4d60-bc5f-3ad1cc483190,1.655757e+09,https://storage.googleapis.com/arize-assets/fi...,"[0.26720312, 0.02652928, 0.0, 0.028591828, 0.0...",drinking,drinking
1,37596b85-c007-4e4f-901d-b87e5297d4b8,1.655757e+09,https://storage.googleapis.com/arize-assets/fi...,"[0.08745878, 0.0, 0.16057675, 0.036570743, 0.0...",fighting,fighting
2,b048d389-539a-4ffb-be61-2f4daa52e700,1.655757e+09,https://storage.googleapis.com/arize-assets/fi...,"[0.9822482, 0.0, 0.037284207, 0.017358225, 0.2...",clapping,clapping
3,3e00c023-49b4-49c2-9922-7ecbf1349c04,1.655757e+09,https://storage.googleapis.com/arize-assets/fi...,"[0.028404092, 0.063946, 1.0448836, 0.65191674,...",fighting,fighting
4,fb38b050-fb12-43af-b27d-629653b5df86,1.655758e+09,https://storage.googleapis.com/arize-assets/fi...,"[0.06121698, 0.5172761, 0.50730985, 0.5771937,...",sitting,sitting


In [22]:
#Define schema
prod_schema = px.Schema(
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="predicted_action",
    embedding_feature_column_names={
        "image_embedding": px.EmbeddingColumnNames(
            vector_column_name="image_vector",
            link_to_data_column_name="url",
        ),
    },
)

In [24]:
#Wrap into inferences
prod_ds = px.Inferences(dataframe=prod_df, schema=prod_schema, name="production")

px.Inferences()

In [25]:
#Launch Phoenix with both inferences
session = px.launch_app(primary=prod_ds, reference=train_ds)

WARNI [phoenix.session.session] Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


/opt/anaconda3/envs/phoenix/lib/python3.12/site-packages/phoenix/server/api/types/EmbeddingDimension.py:425: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prediction_id=dataset[PREDICTION_ID][row_id],
/opt/anaconda3/envs/phoenix/lib/python3.12/site-packages/phoenix/server/api/types/EmbeddingDimension.py:426: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  link_to_data=dataset[self.dimension.link_to_data][row_id],
/opt/anaconda3/envs/phoenix/lib/python3.12/site-packages/phoenix/server/api/types/EmbeddingDimension.py:427: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, int